In [1]:
from pandas import DataFrame, read_csv, to_datetime
import boto3
import os
from decimal import Decimal


In [11]:
import os
from pandas import DataFrame, read_csv

def load_electricity() -> DataFrame:
    current_dir = os.path.dirname(os.path.abspath('src'))
    file_path = os.path.join(current_dir, "data", "electricity_costs.csv")

    return DataFrame(read_csv(file_path))


def load_btc_prices() -> DataFrame:
    dynamodb = boto3.resource("dynamodb")
    table = dynamodb.Table("asics-dynamodb-PricesTable-1LXU143BUOBN")
    response = table.scan()
    items = response["Items"]
    df = DataFrame(items)
    df["Date"] = to_datetime(df["Date"])
    df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")
    df.sort_values(by="Date", inplace=True)
    return df


def load_difficulties() -> DataFrame:
    dynamodb = boto3.resource("dynamodb")
    table = dynamodb.Table("asics-dynamodb-DifficultyTable-DQ308ID3POT6")
    response = table.scan()
    items = response["Items"]
    df = DataFrame(items)
    df.sort_values(by="Date", inplace=True)
    return df


In [12]:
electricity = load_electricity()
btc_prices = load_btc_prices()
difficulties = load_difficulties()

In [14]:
merged_df = electricity.merge(btc_prices, on="Date")
merged_df = merged_df.merge(difficulties, on="Date")

In [16]:

merged_df

,Country,ISO3 Code,Date,Price (EUR/MWhe),ID_x,Price,ID_y,Difficulty
0,Austria,AUT,2018-01-01,-24.33,0da77b6d-c4e3-4754-80a1-5e972569480c,13657.2001953125,771a0908-8ad0-4c6b-8b71-812fda8ad3d3,1931136454487.7
1,Belgium,BEL,2018-01-01,4.63,0da77b6d-c4e3-4754-80a1-5e972569480c,13657.2001953125,771a0908-8ad0-4c6b-8b71-812fda8ad3d3,1931136454487.7
2,Bulgaria,BGR,2018-01-01,18.72,0da77b6d-c4e3-4754-80a1-5e972569480c,13657.2001953125,771a0908-8ad0-4c6b-8b71-812fda8ad3d3,1931136454487.7
3,Switzerland,CHE,2018-01-01,17.09,0da77b6d-c4e3-4754-80a1-5e972569480c,13657.2001953125,771a0908-8ad0-4c6b-8b71-812fda8ad3d3,1931136454487.7
4,Czechia,CZE,2018-01-01,-1.99,0da77b6d-c4e3-4754-80a1-5e972569480c,13657.2001953125,771a0908-8ad0-4c6b-8b71-812fda8ad3d3,1931136454487.7
...,...,...,...,...,...,...,...,...
64635,Romania,ROU,2024-04-30,58.85,11480b3e-42b6-4297-a488-6a58919e9652,60636.85546875,d5c49928-6fc6-4be4-881d-810b243a3f15,86388558925171
64636,Serbia,SRB,2024-04-30,66.52,11480b3e-42b6-4297-a488-6a58919e9652,60636.85546875,d5c49928-6fc6-4be4-881d-810b243a3f15,86388558925171
64637,Slovakia,SVK,2024-04-30,64.68,11480b3e-42b6-4297-a488-6a58919e9652,60636.85546875,d5c49928-6fc6-4be4-881d-810b243a3f15,86388558925171
64638,Slovenia,SVN,2024-04-30,64.08,11480b3e-42b6-4297-a488-6a58919e9652,60636.85546875,d5c49928-6fc6-4be4-881d-810b243a3f15,86388558925171


In [17]:
merged_df = merged_df[merged_df["Country"] == "Austria"]

In [46]:
merged_df["Electicity_day_price"] = merged_df["Price (EUR/MWhe)"] * 24
merged_df[f'Electicity_day_price'] = merged_df[f'Electicity_day_price'].apply(lambda x: Decimal(x))

/var/folders/jf/_21ptb_s1ql4zpjz8yn2_5cw0000gn/T/ipykernel_46061/1447634134.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df["Electicity_day_price"] = merged_df["Price (EUR/MWhe)"] * 24
/var/folders/jf/_21ptb_s1ql4zpjz8yn2_5cw0000gn/T/ipykernel_46061/1447634134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[f'Electicity_day_price'] = merged_df[f'Electicity_day_price'].apply(lambda x: Decimal(x))


In [47]:
from decimal import Decimal

for i in [2, 3, 4, 5, 6]:
    merged_df[f'Cost_{i}_months'] = merged_df['Electicity_day_price'].rolling(window=i).sum()
    merged_df[f'Cost_{i}_months'] = merged_df[f'Cost_{i}_months'].apply(lambda x: Decimal(x))

/var/folders/jf/_21ptb_s1ql4zpjz8yn2_5cw0000gn/T/ipykernel_46061/4213273142.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[f'Cost_{i}_months'] = merged_df['Electicity_day_price'].rolling(window=i).sum()
/var/folders/jf/_21ptb_s1ql4zpjz8yn2_5cw0000gn/T/ipykernel_46061/4213273142.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[f'Cost_{i}_months'] = merged_df[f'Cost_{i}_months'].apply(lambda x: Decimal(x))


In [48]:
merged_df = merged_df.dropna()


In [49]:
merged_df

,Country,ISO3 Code,Date,Price (EUR/MWhe),ID_x,Price,ID_y,Difficulty,Cost_2_months,Cost_3_months,Cost_4_months,Cost_5_months,Cost_6_months,BTC_needed_2_months,BTC_needed_3_months,BTC_needed_4_months,BTC_needed_5_months,BTC_needed_6_months,Electicity_day_price
420,Austria,AUT,2018-01-16,28.54,6b9f0665-71b3-4e35-8403-53892ca98ab4,11490.5,2c949dea-a988-4c5b-a338-6b6e6f85854b,2227847638503.6,1290,1977.36000000000012732925824820995330810546875,2772.239999999999781721271574497222900390625,3783.36000000000012732925824820995330810546875,4924.559999999999490682967007160186767578125,0.004677777294286584569862059963,0.007170271093512031727812905322,0.01005265219094034246689941234,0.01371915930551324888887163077,0.01785736042817980050705045434,684.9600000000000363797880709171295166015625
448,Austria,AUT,2018-01-17,28.65,dcee6374-fd3c-4135-a962-f5a61233dd01,11188.599609375,dc244b35-fc1c-43a8-a40e-a94a7e152081,2227847638503.6,1372.55999999999994543031789362430572509765625,1977.59999999999990905052982270717620849609375,2664.9600000000000363797880709171295166015625,3459.83999999999969077180139720439910888671875,4470.9600000000000363797880709171295166015625,0.005111452907125224744119385914,0.007364639264681212881901280596,0.009924387669298565089615621938,0.01288454364558790716865122590,0.01664998359972649265190305772,687.59999999999990905052982270717620849609375
476,Austria,AUT,2018-01-18,31.04,34303e10-c287-4b32-844c-0c3a1caad113,11474.900390625,c856728b-5c23-4997-859a-7b6ec180fd1b,2227847638503.6,1432.55999999999994543031789362430572509765625,2117.51999999999998181010596454143524169921875,2722.55999999999994543031789362430572509765625,3409.920000000000072759576141834259033203125,4204.8000000000001818989403545856475830078125,0.005201788073800340865541755916,0.007688955633295427020031409891,0.009885924595274094126948402795,0.01238180682736727048980842394,0.01526810639185491196550656139,744.9600000000000363797880709171295166015625
504,Austria,AUT,2018-01-19,40.47,4d266e8a-dde8-4e31-a908-0b15ff49d7ae,11607.400390625,3846ba7b-9ddc-4ff1-93ef-4dadd3ff58d9,2227847638503.6,1716.240000000000009094947017729282379150390625,2403.83999999999969077180139720439910888671875,3088.79999999999972715158946812152862548828125,3693.83999999999969077180139720439910888671875,4381.1999999999998181010596454143524169921875,0.006160724847378986456711787103,0.008628977775324840455496413101,0.01108775399045833572622024181,0.01325964426318137190793558134,0.01572703567178064873092749984,971.279999999999972715158946812152862548828125
532,Austria,AUT,2018-01-20,35.96,4c7593f9-5664-488d-998f-ba9b76a84b1e,12899.2001953125,0d5f2cb3-3014-46e4-94c9-9e01152726e3,2227847638503.6,1834.319999999999936335370875895023345947265625,2579.2799999999997453414835035800933837890625,3266.87999999999965439201332628726959228515625,3951.83999999999969077180139720439910888671875,4556.8800000000001091393642127513885498046875,0.005925173564464427639834388171,0.008331524309472615136540365211,0.01055259224905008302103069716,0.01276513252812655433590278658,0.01471951726658198093045928107,863.0399999999999636202119290828704833984375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64495,Austria,AUT,2024-04-26,85.81,ccd44df7-c0ab-40e2-afc1-d4006d025a42,63755.3203125,4976e979-8a66-465d-a4a4-8f2d6223a0d7,86388558925171,4221.840000000005602487362921237945556640625,6486.960000000004583853296935558319091796875,8838.47999999999956344254314899444580078125,11190,13492.79999999999927240423858165740966796875,0.002759142282365895635847369006,0.004239489326932393695515641939,0.005776302247324702150677567652,0.007313115167717007039491236800,0.008818087608129763413220795096,2059.44000000000005456968210637569427490234375
64524,Austria,AUT,2024-04-27,54.89,085dc676-50b5-448e-9f22-7ebdfec46bb9,63419.140625,8d041841-b918-4177-9e92-c39ef141af05,86388558925171,3376.80000000000609361450187861919403076171875,5539.200000000006184563972055912017822265625,7804.3199999999997089616954326629638671875,101

In [42]:
def add_btc_needed(df: DataFrame) -> DataFrame:
    for i in range(2, 7):
        df[f"BTC_needed_{i}_months"] = df[f"Cost_{i}_months"] / df["Price"]
    return df

In [43]:
merged_df = add_btc_needed(merged_df)

/var/folders/jf/_21ptb_s1ql4zpjz8yn2_5cw0000gn/T/ipykernel_46061/2797257916.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"BTC_needed_{i}_months"] = df[f"Cost_{i}_months"] / df["Price"]
/var/folders/jf/_21ptb_s1ql4zpjz8yn2_5cw0000gn/T/ipykernel_46061/2797257916.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"BTC_needed_{i}_months"] = df[f"Cost_{i}_months"] / df["Price"]
/var/folders/jf/_21ptb_s1ql4zpjz8yn2_5cw0000gn/T/ipykernel_46061/2797257916.py:3: SettingWithCopyWarning: 
A value is tr

In [44]:
merged_df


,Country,ISO3 Code,Date,Price (EUR/MWhe),ID_x,Price,ID_y,Difficulty,Cost_2_months,Cost_3_months,Cost_4_months,Cost_5_months,Cost_6_months,BTC_needed_2_months,BTC_needed_3_months,BTC_needed_4_months,BTC_needed_5_months,BTC_needed_6_months
280,Austria,AUT,2018-01-11,47.55,5e6754d8-9b6e-4691-bf14-0988aab3a612,13405.7998046875,9060b428-5fc1-4672-b8e8-1ead17373fe5,1931136454487.7,89.18999999999999772626324556767940521240234375,120.159999999999996589394868351519107818603515625,150.42000000000001591615728102624416351318359375,173.68000000000000682121026329696178436279296875,209.969999999999998863131622783839702606201171875,0.006653090550316411242517217650,0.008963284679067383928578920460,0.01122051665633585279183290288,0.01295558657673454848570594552,0.01566262386870654666039169743
308,Austria,AUT,2018-01-12,42.13,ceed6d03-59e7-4895-976e-e2e75c716eab,13980.599609375,52991cba-685f-4389-bd29-ade395ffd32f,1931136454487.7,89.68000000000000682121026329696178436279296875,131.31999999999999317878973670303821563720703125,162.289999999999992041921359486877918243408203125,192.549999999999982946974341757595539093017578125,215.81000000000000227373675443232059478759765625,0.006414603272084489219290578701,0.009393016298953334260148772733,0.01160822886960963007547480850,0.01377265678010557756288825782,0.01543639085803471819475190974
336,Austria,AUT,2018-01-13,33.12,6efba0be-2c61-43f5-ab53-200079a14dac,14360.2001953125,4c8cfea2-7a6b-4b07-98e7-9694fc85fc36,2227847638503.6,75.25,122.799999999999982946974341757595539093017578125,164.43999999999999772626324556767940521240234375,195.409999999999996589394868351519107818603515625,225.669999999999987494447850622236728668212890625,0.005240177642130875911986831551,0.008551412816660086015031217667,0.01145109384015948469172531662,0.01360774901061520855932451822,0.01571496197341760400700004652
364,Austria,AUT,2018-01-14,28.64,b4777f0a-50e9-4897-a955-d83d13bec955,13772,ef319dc6-85d7-4a05-b4a6-354f9a7b6517,2227847638503.6,61.7599999999999980104803398717194795608520507...,103.890000000000014779288903810083866119384765625,151.43999999999999772626324556767940521240234375,193.07999999999998408384271897375583648681640625,224.05000000000001136868377216160297393798828125,0.004484461225675283038809202721,0.007543566656985188409765386568,0.01099622422306128359906064810,0.01401975021783328377024707515,0.01626851582921870544355821755
392,Austria,AUT,2018-01-15,25.21,1a81e442-283e-4b23-ac7c-e4ab58ac5505,13819.7998046875,be70571d-d859-4b5d-b23d-fd9a265bcf94,2227847638503.6,53.8500000000000014210854715202003717422485351...,86.969999999999998863131622783839702606201171875,129.099999999999994315658113919198513031005859375,176.650000000000005684341886080801486968994140625,218.289999999999992041921359486877918243408203125,0.003896583218357097154550428336,0.006293144707530487022437363012,0.009341669331288787506548533538,0.01278238487507486009908974798,0.01579545312414430240317192887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64495,Austria,AUT,2024-04-26,85.81,ccd44df7-c0ab-40e2-afc1-d4006d025a42,63755.3203125,4976e979-8a66-465d-a4a4-8f2d6223a0d7,86388558925171,175.909999999999996589394868351519107818603515625,270.28999999999979308995534665882587432861328125,368.26999999999998181010596454143524169921875,466.24999999999994315658113919198513031005859375,562.200000000000045474735088646411895751953125,0.002759142282365895635847369006,0.004239489326932393695515641939,0.005776302247324702150677567652,0.007313115167717007039491236800,0.008818087608129763413220795096
64524,Austria,AUT,2024-04-27,54.89,085dc676-50b5-448e-9f22-7ebdfec46bb9,63419.140625,8d041841-b918-4177-9e92-c39ef141af05,86388558925171,140.69999999999998863131622783839702606201171875,230.799999999999812416717759333550930023193359375,325.1799999999999499777914024889469146728515625,423.1599999999999681676854379475116729736328125,521.1399999999999863575794734060764312744140625,0.002218573109212641410359953578,0.003639279840840634418746789181,0.00